In [1]:
from trainer import Trainer
from meta_featured_trainer import MetaFeatureTrainer
from resnet import fineTuneResNet50,lessFilterResNet50, paperResNet18, nnResNet, get_ensemble_resnet
from senet import senetXX_generic
from swwae_trainer import SWWAETrainer
from swwae import get_swwae
import pandas as pd
import numpy as np
%matplotlib notebook

In [2]:
from sklearn.model_selection import KFold
fold_num = 5

In [3]:
import torch
from torch.optim import lr_scheduler
import gc

In [4]:
test_ids = pd.read_csv('Data/test_ids.csv')['id']

In [5]:
gc.enable()
gc.collect()

7

In [6]:
test_df = pd.read_json('Data/polar_split_no_denoise_processed_test.json', dtype={'inc_angle': str})
test_df.inc_angle = pd.to_numeric(test_df.inc_angle, errors='coerce')

#test_df.inc_angle = (test_df.inc_angle / 0.3).astype(np.int)

#test_df.reset_index(drop=True, inplace=True)
print(test_df.columns)
test_df.head(3)

Index(['band_mixed', 'inc_angle', 'is_iceberg'], dtype='object')


,band_mixed,inc_angle,is_iceberg
0,"[[[0.3602381271, 0.3185865893, 0.6397618729, 0...",34.966400,0
1,"[[[0.23705839, 0.2707230281, 0.76294161, 0.729...",32.615072,0
10,"[[[0.3381528741, 0.2572200913, 0.6618471259, 0...",35.782900,0


In [7]:
test_df.sort_index(inplace=True)
test_df.head(5)

,band_mixed,inc_angle,is_iceberg
0,"[[[0.3602381271, 0.3185865893, 0.6397618729, 0...",34.966400,0
1,"[[[0.23705839, 0.2707230281, 0.76294161, 0.729...",32.615072,0
2,"[[[0.381044323, 0.39847112970000004, 0.6189556...",37.505433,0
3,"[[[0.4048886681, 0.28734042120000003, 0.595111...",34.473900,0
4,"[[[0.269489719, 0.2710927935, 0.730510281, 0.7...",43.918874,0


In [8]:
submission = pd.DataFrame()
submission['id'] = test_ids
submission['is_iceberg'] = np.zeros((len(test_ids),))

In [9]:
def predict_with_oof_models(seed=0):
    global fold_num
    global test_ids
    
    final_predictions = []
    for counter in range(fold_num):

        is_transfer_learning = False
        model = get_ensemble_resnet()
        #model = nnResNet(meta_features=1)  #dpn92(num_classes=1) #lessFilterResNet50()
        model_name= 'Trained_model/ens_resnet_cpl_' + str(counter+1) + '.db'
        # model_name = 'Trained_model/nn_resnet_soft_pl_n_valid_' + str(counter+1)+ '.db'
        # testing snap shot for essemble
        optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=0.001)

        tr = Trainer(
                model,
                optimizer,
                epochs=1800,
                milestones=[300, 1100, 1600],
                gamma=0.2,
                batch_size=128, 
                use_cuda=True, 
                gpu_idx=0,
                best_model_name = model_name,
                seed=seed,
                verbose=0)

        tr.load_checkpoint()
        
        data_augmentation_args = {
            'mirror': False, # not useful here
            'rotate': True,
            'scale': True,
            'translation': True
        }
        
        predictions = tr.test(test_df, is_transfer_learning=is_transfer_learning, is_augment=True, data_augmentation_args=data_augmentation_args)
        predictions = predictions.reshape((predictions.shape[0],))
        final_predictions.append(predictions)

        del model, optimizer, tr
        
    return final_predictions

In [10]:
repeat_count = 40
for i in range(repeat_count):
    print(i, 'round')
    predictions = predict_with_oof_models(seed=i)
    for j, p in enumerate(predictions):
        submission['is_iceberg_r' + str(i) + '_p' + str(j) ] = p

0 round
gpu: 0  available: True
epoch= 106 best_loss= 0.05816717385314405
gpu: 0  available: True
epoch= 135 best_loss= 0.07412643822220465
gpu: 0  available: True
epoch= 98 best_loss= 0.07048591789789498
gpu: 0  available: True
epoch= 80 best_loss= 0.07901024012826383
gpu: 0  available: True
epoch= 110 best_loss= 0.0714154920152699
1 round
gpu: 0  available: True
epoch= 106 best_loss= 0.05816717385314405
gpu: 0  available: True
epoch= 135 best_loss= 0.07412643822220465
gpu: 0  available: True
epoch= 98 best_loss= 0.07048591789789498
gpu: 0  available: True
epoch= 80 best_loss= 0.07901024012826383
gpu: 0  available: True
epoch= 110 best_loss= 0.0714154920152699
2 round
gpu: 0  available: True
epoch= 106 best_loss= 0.05816717385314405
gpu: 0  available: True
epoch= 135 best_loss= 0.07412643822220465
gpu: 0  available: True
epoch= 98 best_loss= 0.07048591789789498
gpu: 0  available: True
epoch= 80 best_loss= 0.07901024012826383
gpu: 0  available: True
epoch= 110 best_loss= 0.071415492015

gpu: 0  available: True
epoch= 98 best_loss= 0.07048591789789498
gpu: 0  available: True
epoch= 80 best_loss= 0.07901024012826383
gpu: 0  available: True
epoch= 110 best_loss= 0.0714154920152699
25 round
gpu: 0  available: True
epoch= 106 best_loss= 0.05816717385314405
gpu: 0  available: True
epoch= 135 best_loss= 0.07412643822220465
gpu: 0  available: True
epoch= 98 best_loss= 0.07048591789789498
gpu: 0  available: True
epoch= 80 best_loss= 0.07901024012826383
gpu: 0  available: True
epoch= 110 best_loss= 0.0714154920152699
26 round
gpu: 0  available: True
epoch= 106 best_loss= 0.05816717385314405
gpu: 0  available: True
epoch= 135 best_loss= 0.07412643822220465
gpu: 0  available: True
epoch= 98 best_loss= 0.07048591789789498
gpu: 0  available: True
epoch= 80 best_loss= 0.07901024012826383
gpu: 0  available: True
epoch= 110 best_loss= 0.0714154920152699
27 round
gpu: 0  available: True
epoch= 106 best_loss= 0.05816717385314405
gpu: 0  available: True
epoch= 135 best_loss= 0.0741264382

In [16]:
submission['is_iceberg_max'] = submission.iloc[:, 2:repeat_count*fold_num+2].max(axis=1)
submission['is_iceberg_min'] = submission.iloc[:, 2:repeat_count*fold_num+2].min(axis=1)
submission['is_iceberg_mean'] = submission.iloc[:, 2:repeat_count*fold_num+2].mean(axis=1)
submission['is_iceberg_median'] = submission.iloc[:, 2:repeat_count*fold_num+2].median(axis=1)

In [17]:
submission['is_iceberg'] = submission['is_iceberg_mean']
submission[['id', 'is_iceberg']].to_csv('Submissions/submission_ensemble_resnet_cpl_oof_augment_avg.csv', 
                                        index=False, float_format='%.15f')

In [18]:
cutoff_hi=.1
cutoff_lo=.9
submission['is_iceberg'] = np.where(np.all(submission.iloc[:,2:repeat_count*fold_num+2] > cutoff_lo, axis=1), 
                                    submission['is_iceberg_max'], # all > lo
                                    np.where(np.all(submission.iloc[:,2:repeat_count*fold_num+2] < cutoff_hi, axis=1),
                                             submission['is_iceberg_min'], # all < hi 
                                             submission['is_iceberg_median'] # use the best
                                            )
                                   )

In [19]:
submission[['id', 'is_iceberg']].to_csv('Submissions/submission_ensemble_resnet_cpl_oof_augment_minmax_median.csv'.format(repeat_count)
                  , float_format="%.15f", index=False) 

In [20]:
submission.to_csv('Data/submission_ensemble_resnet_cpl_oof_augment_origin.csv'.format(repeat_count)
                  , float_format="%.15f", index=False)